# Naïve Bayes classifier - An introduction

## 1. The "Bayes" part

In the Naive Bayes classifier, the Bayes part comes from the Bayes theorem, and a link to Bayesian statistics. Now, we won't deep dive into the details of Bayesian statistics [1] but you can take the following principle from it: the Bayesian approach to probability allows you to incorporate prior information or expectations you have about your data and update them as you see new observations, to generate a final expectation over your hypothesis.   

Unlike frequentist approaches, which assume that data is generated from a distribution with fixed but unknown parameters (which are estimated), Bayesian approaches treat the parameters themselves as random variables, drawn from a specific probability distribution.

### 1.1 Bayes’ Theorem

The Bayes' theorem is a way of representing the conditional probability of two events A and B.

First, let's define some variables:

* $P(A)$: probability of event A ocurring
* $P(B)$: probability of event B ocurring
* $P(A|B)$: probability of event A ocurring knowing that B ocurred
* $P(B|A)$: probability of event B ocurring knowing that A ocurred
* $P(A\cap B)$: probability of event A and event B ocurring together, i.e., the joint probability of both events

Now, starting from the conditional probabilities, we know that:

$$P(A\cap B) = P(A|B)P(B) $$ 

In the same way:

$$P(A\cap B) = P(B|A)P(A) $$ 

From this, the Bayes theorem derives the following:

$$ P(A|B)P(B) = P(B|A)P(A)  \Leftrightarrow  P(A|B) = \frac{P(B|A)P(A)}{P(B)} $$ 

Now, if you think of event A as a hypothesis we're interested in testing, and B as the data we observe, we could write the following:

$$ P(hyp|data) = \frac{P(data|hyp)P(hyp)}{P(data)} $$ 

Going a bit further with this you can describe:

* $P(hyp)$: our **prior** knowledge about the hypothesis
* $P(data)$: overall probability of our data, independent of any hypothesis
* $P(data|hyp)$: probability of this data ocurring if our hypothesis is true, usually called the **likelihood** of our data
* $P(hyp|data)$: how likely our hypothesis is to be true given the data we're observing, usually called the **posterior** probability

$P(hyp)$ can be seen as your **prior** belief in our hypothesis, while $P(hyp|data)$ can be seen as an updated belief, or **posterior** belief after observing some data.

![seashell](media/seashell.png)

### 1.2 The "Naïve" part

Now that you got the gist of Bayes' theorem, let's dive into why it's called naïve. The theorem we presented above seems quite simple right?

$$ P(hyp|data) = \frac{P(data|hyp)P(hyp)}{P(data)} $$ 

But if you take a closer look, the overal notion of observed `data` can be quite complex. By data, we usually refer to all the events that we are using to predict the probability of our hypothesis. _Do you see where I'm going with this?_

That's right, I'm talking about features. And if we expand that to what we typically have - multiple features - then each feature can be seen as one event, and thus we can rewrite the equation as follows:


$$ P(hyp|\{x_1 \cap x_2 \cap ... \cap x_n\}) = \frac{P(\{x_1 \cap x_2, ... \cap x_n\}|hyp)P(hyp)}{P(\{x_1 \cap x_2 \cap ... \cap x_n\})} $$ 

This is where the naïve part comes in. We are going to make an assumption, and a pretty big one. We're going to assume that our features are independent of each other. With this **independence assumption** we get the following:

$$ P(x_1 \cap x_2, ... \cap x_n) = P(x_1)P(x_2) ... P(x_n) $$

which is also valid for the conditional probability

$$ P(x_1 \cap x_2, ... \cap x_n | hyp) = P(x_1|hyp)P(x_2|hyp) ... P(x_n|hyp) $$

Putting all of this together we get:

$$ P(hyp|\{x_1 \cap x_2 \cap ... \cap x_n\}) = \frac{P(x_1|hyp)P(x_2|hyp) ... P(x_n|hyp) * P(hyp)}{P(x_1)P(x_2) ... P(x_n))} $$ 

## 2. A simple example

Imagine that we have the following hypothesis over a piece of fruit taken from a bowl of fruit. Our hypothesis will be that the piece of fruit is an orange:

$H_0$: fruit is an orange

And we define some features that we observe 

* feature 1 ($x_1$), color  &ensp;&emsp;-> &emsp; $x_1$=orange
* feature 2 ($x_2$), shape &emsp;-> &emsp; $x_2$=round

Then by definition:

$$ P(H_0|\{x_1 \cap x_2\}) = \frac{P(x_1|H_0)P(x_2|H_0)P(H_0)}{P(x_1)P(x_2)} $$ 

### 2.1 Prior

As mentioned before, $P(H_0)$ is our prior belief in the hypothesis. If I only buy oranges and apples, and I buy them in the same quantities, for example, I could say my prior probability that I'll get an orange from my fruit bowl is the same as that - 0.5.

In [1]:
prior = 0.5

### 2.2 Likelihoods

If I have some previous data that I've seen:

| observation | color | shape | is_orange | 
|---|-------|-------|-------|
| 1 | orange  | round | yes | 
| 2 | orange  | round | yes | 
| 3 | orange  | round | no | 
| 4 | red  | round | no | 
| 5 | green  | round | yes | 
| 6 | red  | not round | no | 

Then I can extract my likelihoods from here.

$$ P(x_1=orange|H_0) = \frac{count(x_1=orange, H_0=yes)}{count(H_0=yes)}= \frac{2}{3} \approx 0.666 $$

In plain words, for the feature color $(x_1)$, the table has 2 instances where $ x_1 = \text{orange} $ and $ H_0 = \text{yes} $ (observations 1 and 2) out of 3 total instances where $ H_0 = \text{yes} $ (observations 1, 2, and 5).


In the same way, we calculate it for the feature shape $(x_2)$:

$$ P(x_2=round|H_0) = \frac{count(x_2=round, H_0=yes)}{count(H_0=yes)}= \frac{3}{3}=1.0$$


Let's put this experiment into code:

In [2]:
data = [
    ("orange", "round", "yes"),
    ("orange", "round", "yes"),
    ("orange", "round", "no"),
    ("red", "round", "no"),
    ("green", "round", "yes"),
    ("red", "not round", "no"),
]

In [3]:
def likelihood(data, feat_index, feat_value, hyp):
    
    count_hyp = 0
    count_feat_hyp = 0
    for row in data:
        if row[-1] == hyp:
            count_hyp += 1
            if row[feat_index] == feat_value:
                count_feat_hyp += 1
    
    return 1.0*count_feat_hyp/count_hyp

In [4]:
p_x_1_h_0 = likelihood(data, 0, "orange", "yes")
p_x_2_h_0 = likelihood(data, 1, "round", "yes")

print(p_x_1_h_0)
print(p_x_2_h_0)

0.6666666666666666
1.0


### 2.3 Evidence

Finally, we'll do $P(x_1)$ and $P(x_2)$, although it is slightly less important, because it's just a normalizing constant for multi-class problems. Let's extract it for our current dataset:

For $P(x_1=orange)$: out of 6 observations, 3 have $x_1 = orange$.

$$P(x_1 = \text{orange}) = \frac{3}{6} = 0.5$$

For $P(x_2=round)$: out of 6 observations, 5 have $x_2 = round$.

$$P(x_2 = \text{round}) = \frac{5}{6} \approx 0.833$$

As you can see, the evidence terms reflect the overall probability of observing a feature in the entire dataset.

Let's put this into code:

In [5]:
def prob_evidence(data, feat_index, feat_value):
    
    count_rows = 0
    count_feat = 0
    for row in data:
        count_rows += 1
        if row[feat_index] == feat_value:
            count_feat += 1
    
    return 1.0*count_feat/count_rows

In [6]:
p_x_1 = prob_evidence(data, 0, "orange")
p_x_2 = prob_evidence(data, 1, "round")

print(p_x_1)
print(p_x_2)

0.5
0.8333333333333334


### 2.4 Posterior probability
Now we can infer our final posterior probability for the hypothesis by pluging all these values into Bayes' theorem:

$$
P(H_0 \mid x_1, x_2) = \frac{P(x_1 \mid H_0) P(x_2 \mid H_0) P(H_0)}{P(x_1) P(x_2)}
$$

Substitute the values:

- $P(x_1 \mid H_0) = 0.666$
- $P(x_2 \mid H_0) = 1.0$
- $P(H_0) = 0.5$
- $P(x_1) = 0.5$
- $P(x_2) = 0.833$

$$
P(H_0 \mid x_1, x_2) = \frac{0.666 \times 1.0 \times 0.5}{0.5 \times 0.833} = 0.7999999999999999
$$

In [7]:
p_h_0_X = (p_x_1_h_0 * p_x_2_h_0 * prior)/(p_x_1*p_x_2)
print(p_h_0_X)

0.7999999999999999


### 2.5 Effect of changing the prior
If my prior had been different, let's say that I buy oranges very rarely, *$P(H_0) = 0.1$*, we would get a different result:

In [8]:
prior = 0.1
p_h_0_X = (p_x_1_h_0 * p_x_2_h_0 * prior)/(p_x_1*p_x_2)
print(p_h_0_X)

0.16


This shows how the posterior probability is influenced by the prior belief.

By now, the idea that we're combining prior knowledge or beliefs that we have with actual observations should be more or less obvious to you. The following image from this [article](https://towardsdatascience.com/what-is-bayesian-inference-4eda9f9e20a6) shows you a visual interpretation of this:

![bayes_charts](media/bayes_visualization.png)

## 3. Applying Naïve Bayes to text classification

Now for the interesting part - applying it to text classification!

Imagine that we want to classify a particular review as:

* $C_0$: positive review
* $C_1$: negative review
* $C_2$: neutral review

According to our formulas we have:

$$P(C_i|D_j) = \frac{P(D_j|C_i)*P(C_i)}{P(D_j)}$$

Where: 

* $P(C_i)$ is the overall probability of the class, the **prior**;
* $P(D_j)$ is the overall probability of the document;
* $P(D_j|C_i)$ is the **likelihood** of a document showing up for a particular class.

To make things simpler notice that $P(D_j)$ is constant across the two classes, i.e., when comparing among classes, the value doesn't change. Since we're only interested in knowing which class has higher probability, we can just say that $P(C_i|D_j)$ is proportional to the following quantity:

$$P(D_j|C_i)*P(C_i)$$

And so this is the formula we will use to compute our **comparative probabilities** between classes. 

Now, a document is a set of words, which we consider our features. Using (and abusing) the independence assumption of Naïve Bayes, the likelihood of a document showing in a particular class is just the likelihood of its set of words showing up in that class:

$$P(D_j|C_i) = P(w_1|C_i) * P(w_2|C_i) * P(w_3|C_i) * ... * P(w_n|C_i)$$

And our final formula becomes:

$$P(C_i|D_j) = [P(w_1|C_i) * P(w_2|C_i) * P(w_3|C_i) * ... * P(w_n|C_i)]*P(C_i)$$


In an analogous way to our previous fruit example, we consider $P(w_j|C_i)$ the likelihoods that we can calculate for the classes, and $P(C_i)$ the prior on all the classes.

So, similar to the example we did above:

### 3.1 Training

* estimate $P(w_k|C_i)$ for all words, based on counts of the words in documents
* set or estimate $P(C_i)$, either from dataset, previous knowledge or just assign equal probabilities

### 3.2 Predicting

* Compute $P(C_i|D_j) = [P(w_1|C_i) * P(w_2|C_i) * P(w_3|C_i) * ... * P(w_n|C_i)] * P(C_i)$
* Choose the class satisfying $C = argmax_i P(C_i|D_j)$


This is exactly what `MultinomialNB` does for you, and the reason why it's a good baseline follows (quoting directly from [3]):

> Because of the independence assumption, naive Bayes classifiers can quickly learn to use high dimensional feature spaces with limited training data compared to more sophisticated methods. This can be useful in situations where the dataset is small compared to the number of features, such as images or texts.

There are other ways of handling the dimensionality curse, as you'll see in the next BLU, but now you understand why Naïve Bayes is such an interesting method in these cases.

## 4. References

1. [MIT OpenCourseWare - Statistics for Applications: 17. Bayesian Statistics](https://www.youtube.com/watch?v=bFZ-0FH5hfs)
2. [What Is Bayesian Inference?](https://towardsdatascience.com/what-is-bayesian-inference-4eda9f9e20a6)
3. [Understanding mathematics behind naive bayes](https://shuzhanfan.github.io/2018/06/understanding-mathematics-behind-naive-bayes/)
